In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [1]:
import torch
torch.cuda.empty_cache()

In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
pip install -U langchain-huggingface

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install protobuf==3.20.0 grpcio==1.39.0

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 1.0 MB 411 kB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 449 kB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.65.1
    Uninstalling grpcio-1.65.1:
      Successfully uninstalled grpcio-1.65.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.63.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
chromadb 0.5.5 requires grpcio>=1.58.0, but you have grpcio 1.39.0 which is incompatible.
Note: you may need to restart the ker

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from os import listdir
from os.path import isfile, join

# some comment

embeddings = HuggingFaceEmbeddings(
    model_name = 'emilyalsentzer/Bio_ClinicalBERT',
    model_kwargs={'device': 'cuda'}
)
 
text = "This is a test document."
query_result = embeddings.embed_query(text)




/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/root/miniconda3/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name emilyalsentzer/Bio_ClinicalBERT. Creating a new one with mean pooling.


In [4]:
#onlyfiles = [f for f in listdir('top_1000_txt') if isfile(join('top_1000_txt', f))]
onlyfiles=['note_1227.txt']
raw_documents = []
for file in onlyfiles:
    print(file)
    raw_doc = TextLoader(f'23 txt/{file}').load()
    raw_documents.extend(raw_doc)

text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=24)

note_1227.txt


In [5]:
#onlyfiles = [f for f in listdir('top_1000_txt') if isfile(join('top_1000_txt', f))]
onlyfiles=['note_1227.txt']
raw_documents = []
for file in onlyfiles:
    print(file)
    raw_doc = TextLoader(f'23 txt/{file}').load()
    raw_documents.extend(raw_doc)

text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=24)

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("johnsnowlabs/JSL-MedLlama-3-8B-v2.0")
model = AutoModelForCausalLM.from_pretrained(
    "johnsnowlabs/JSL-MedLlama-3-8B-v2.0",
     device_map="auto"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [7]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# 设置 Hugging Face 管道
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=3000,
    truncation=True,
    repetition_penalty=1.15
)

# 创建本地 LLM
local_llm = HuggingFacePipeline(pipeline=pipe)

# 创建检索器
retriever = db.as_retriever(search_kwargs={"k": 1})

# 创建 QA 链
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True
)

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [8]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [9]:
llm=local_llm
template = """You are an expert clinical assistant. You will receive a collection of clinical notes. Answer the question based only on the following context:

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [10]:
chain.invoke('What allergies did the patient have?')

'Allergic to'

In [10]:
#1227
from datasets import Dataset
questions= [
    "What will make patient return to the hospital?",
    "What was patient found in left parietal?",
    "What was patient diagnosed with?",
    "What dose patient present from nursing home?",
    "What was patient started on?",
    "Does the patient have diabetes?",
    "Did the patient have hypertension?",
    "Does the 62-year-old man have drug allergies?",
    "How old is the man with coronary artery disease and cirrhosis?",
    "What was the troponin level of the 74-year-old man?",
    "What was the left effusion noted with in the 74-year-old man?",
    "When was the 62-year-old man diagnosed with HIT and Afib?",
    "When did the 74-year-old man have an episode of melena?",
    "Where was the embolic stroke located in the 62-year-old man?",
    "Where did the 62-year-old man present from?",
    "What was the INR level for the 74-year-old man?",
    "When should the 43-year-old female see her cardiologist next?",
    "What should patient do if he experiences chest pain or shortness of breath?",
    "How many patients have diabetes?",
    "What was the oxygen saturation level for the 74-year-old man?"


]


ground_truths = [
    ["The patient will return to the hospital if experiencing chest pain, shortness of breath, high fever, or any mental status change."],
    ["He was found to have an embolic stroke in left parietal, left internal capsule region."],
    ["He was diagnosed with HIT and Afib."],
    ["Tachypnea for 2 days, intermittent fevers."],
    ["Dialysis."],
    ["Yes."],
    ["Yes."],
    ["No known allergies to drugs."],
    ["74-year-old."],
    ["troponin 1.5."],
    ["Bibasilar atelectasis, no evidence of pneumonia or congestive heart failure."],
    ["During post op period he was diagnosed with HIT and Afib."],
    ["On the night of admission."],
    ["Left parietal, left internal capsule region."],
    ["A nursing home."],
    ["1.4."],
    ["One week."],
    ["Return pt to hospital if patient experiences chest pain, shortness of breath or develops high fever."],
    ["2/Two patients."],
    ["100% on room air."]
]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('1227txt')#22m11s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#1031
from datasets import Dataset
questions= [
    "What is the 57-year-old patient's basic information?",
    "How many patients have a history of hypertension?",
    "Why did the patient come to the hospital?",
    "When did the patient’s chest pain begin?",
    "What was the left ventricular ejection fraction found during the echocardiogram?",
    "Did the patient have no known drug allergies?",
    "What did the patient use tobacco, alcohol, and other drug?",
    "How many patients have a history of hypertension and hypercholesterolemia?",
    "How many medications were given to the 35-year-old male patient in the ED?",
    "How many patients reported no known drug allergies?",
    "How many patients had elevated liver enzymes",
    "How many patients were prescribed anticoagulants upon discharge?",
    "What is the gender of the 57-year-old patient?",
    "What symptom did the 57-year-old female present with?",
    "At what time did the 57-year-old female arrive at the Emergency Room?",
    "What was the 57-year-old female's blood pressure on admission?",
    "What percentage lesion was found in the proximal LAD of the 57-year-old female?",
    "Was the 57-year-old female started on heparin?",
    "What anticoagulant was the 57-year-old female transitioned to?",
    "What is the gender of the 35-year-old patient?"
]


ground_truths = [
    ["The patient was a 57-year-old female with a history of hypertension and hypercholesterolemia but no prior coronary artery disease history who presented with chest pain."],
    ["2/Two patients."],
    ["She came due to chest pain that started at 4:00 p.m."],
    ["At 4:00 p.m. on the day of admission."],
    ["She was found to have an ejection fraction of 0-35%."],
    ["Yes"],
    ["The patient reported no tobacco use and denied any alcohol or other drug use."],
    ["1"],
    ["Three medications."],
    ["2/Two patients."],
    ["2/Two patients."],
    ["2/Two patients."],
    ["Female."],
    ["Chest pain."],
    ["At 8:00 p.m."],
    ["146/88."],
    ["90%."],
    ["Yes."],
    ["Coumadin"],
    ["Male."]
    
]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('1031txt')#20m14

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#1025
from datasets import Dataset
questions= [
    "What did the chest x-ray show?",
    "What is the patient’s code status?",
    "What did the patient undergo?",
    "What are the patient's primary medical problems?",
    "What imaging studies were performed?",
    "When was the patient made DNR/DNI?",
    "What did serial chest x-ray show?",
    "What is the age of the female patient mentioned in the document?",
    "What was the patient's blood pressure upon examination?",
    "What was the heart rate at the time of initial examination?",
    "What was the pCO2 level?",
    "What was the patient’s initial temperature?",
    "What did the EKG show?",
    "What was the patient's alcohol and tobacco use history?",
    "What did spinal imaging reveal?",
    "What was the newborn’s bilirubin level on day six?",
    "What was the result of the newborn’s car seat screening test?",
    "How many types of antibiotics were administered to the newborn?",
    "How many types of follow-up visits were planned for the myocardial infarction patient?",
    "How many patients took Aspirin?"
]


ground_truths = [
    ["Moderate cardiomegaly with some bilateral opacifications in the left upper and right upper lobes with multifocal pneumonia."],
    ["he patient was made DNR/DNI after multiple discussions with her family."],
    ["The patient underwent a sleep and swallow study and modified barium study."],
    ["Multiple medical problems including atrial fibrillation, chronic obstructive pulmonary disease, coronary artery disease, congestive heart failure."],
    ["Chest X-rays, chest CT Scan, Spinal Imaging, MRI."],
    ["the patient was made DNR/DNI after multiple discussions with her family, as well as proxyholder."],
    ["Serial chest x-ray showed moderate cardiomegaly with some bilateral opacifications in the left upper and right upper lobes with multifocal pneumonia."],
    ["81 years old."],
    ["92/64."],
    ["96 to 120."],
    ["pCO2 of 50."],
    ["98.6 degrees Fahrenheit."],
    ["EKG showed atrial fibrillation at 120, with poor R-wave progression with left axis deviation, pseudochanges in T waves in V1 through V3."],
    ["Denies alcohol and tobacco use."],
    ["Lytic destruction of the T10 vertebral body posterior elements with soft tissue density minimally enhancing mass lesion."],
    ["Total 5.5, direct 0.3."],
    ["The infant passed in both ears."],
    ["2/two"],
    ["2/two"],
    ["1/one patient."]]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('1025txt')#10m35s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#1019
from datasets import Dataset
questions= [
    "What is the patient's past medical history?",
    "How many patients had a family history of stroke?",
    "What is the patient's vital signs on arrival at the hospital?",
    "What is the patient's diagnosis?",
    "Who found the patient unresponsive?",
    "Where was the patient transferred to after being found unresponsive?",
    "What was the patient's condition upon arrival at the hospital?",
    "What condition did another sister of the patient have?",
    "What is the patient's pulse rate on arrival?",
    "What was the left pupil's measurement with light exposure?",
    "Was the right corneal reflex present?",
    "What was the sodium level in the electrolytes?",
    "What was the potassium level in the electrolytes?",
    "What is the size of the left thalamus hemorrhage?",
    "Was a Foley catheter placed in the patient?",
    "What medications were maintained for the patient for comfort?",
    "Was the patient a candidate for neurosurgical intervention?",
    "What are the medical conditions of the patient's sisters?",
    "What is the age of the patient?",
    "What are the results of the patient’s electrolyte and renal function tests?"
]


ground_truths = [
    ["The patient has a history of hypertension and hypercholesterolemia."],
    ["1/One patient."],
    ["Temperature 97.0, blood pressure 133/77, pulse 51, respiratory rate 14."],
    ["The patient is an 83-year-old female with a history of hypertension, hypercholesterolemia, who presented with a large intraparenchymal cerebral hemorrhage complicated by obstructive hydrocephalus."],
    ["A neighbor."],
    ["Hospital."],
    ["Unconscious, with a flaccid right side, staring to the left, and incontinent of urine."],
    ["Multiple sclerosis."],
    ["51."],
    ["3.0 mm, decreasing to 2.5 mm."],
    ["No, she had a left corneal reflex, and no right corneal reflex."],
    ["142."],
    ["Potassium|.0"],
    ["5.2 cm x 3 cm."],
    ["Yes, it remained in place."],
    ["Lorazepam, Morphine, Tylenol."],
    ["No."],
    ["One sister has a brain hemorrhage status post evacuation, and another sister had multiple sclerosis."],
    ["83 years old."],
    ["Sodium 142, potassium|.0, chloride 106, bicarb 21, BUN 15, creatinine 0.6, glucose|45."]]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('1019txt')#16m19s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
dataset
ragas_input_df = dataset.to_pandas()
display(ragas_input_df.head())

,question,answer,contexts,ground_truths
0,What is the patient's past medical history?,Human: You are an expert clinical assistant. Y...,[Patient is an 83-year-old female with a histo...,[The patient has a history of hypertension and...
1,How many patients had a family history of stroke?,Human: You are an expert clinical assistant. Y...,[Patient is an 83-year-old female with a histo...,[1/One patient.]
2,What is the patient's vital signs on arrival a...,Human: You are an expert clinical assistant. Y...,[Patient is an 83-year-old female with a histo...,"[Temperature 97.0, blood pressure 133/77, puls..."
3,What is the patient's diagnosis?,Human: You are an expert clinical assistant. Y...,[Patient is an 83-year-old female with a histo...,[The patient is an 83-year-old female with a h...
4,Who found the patient unresponsive?,Human: You are an expert clinical assistant. Y...,[Patient is an 83-year-old female with a histo...,[A neighbor.]


In [10]:
#832
from datasets import Dataset
questions= [
    "What cardiovascular conditions does the patient have?",
    "What was the patient’s heart rate during the examination?",
    "Does the patient have a history of diabetes?",
    "What is the patient’s blood pressure on examination?",
    "What is the patient’s baseline creatinine level? ",
    "What is the patient's primary language?",
    "Does the patient have a history of myocardial infarction?",
    "What heart procedure did the patient have in addition to myocardial infarction?",
    "Has the patient had coronary artery bypass surgery?",
    "How often should the patient take Percocet for pain?",
    "Did the patient have any cyanosis, clubbing, or edema in the extremities?",
    "Was there any history of penicillin-induced rash?",
    "Was Protonix prescribed to the patient?",
    "How many patients were on Lipitor?",
    "How many patients had a history of hepatitis?",
    "How many patients had a history of smoking?",
    "How many patients had their INR tested?",
    "How many patients had a history of hypercholesterolemia?",
    "What was the patient’s platelet count in preoperative laboratories?",
    "Was there a family history of CVA?"
]


ground_truths = [
    ["Coronary artery disease, myocardial infarction, percutaneous transluminal coronary angioplasty with a stent."],
    ["72."],
    ["Yes, the patient has a history of insulin-dependent diabetes mellitus."],
    ["139/65"],
    ["1.3"],
    ["Spanish."],
    ["Yes."],
    ["Percutaneous transluminal coronary angioplasty with a stent."],
    ["Yes."],
    ["Percocet 5/325-mg tablets one to two tablets by mouth q.4h. as needed (for pain)"],
    ["No."],
    ["Yes."],
    ["Yes."],
    ["1/one patient"],
    ["1/one patient"],
    ["2/two patients"],
    ["2/two patients"],
    ["2/two patients"],
    ["303,000"],
    ["Yes."]

]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('832txt')#10m41s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
#826
from datasets import Dataset
questions= [
    "Does the patient have any known drug allergies?",
    "Did the elderly female patient have a history of smoking?",
    "What was the patient's glucose level at 01:55 AM?",
    "What was the baby’s birth weight?",
    "Has the patient got a history of diabetes?",
    "How many doses of surfactant did the baby receive?",
    "Did the patient take imaging studies?",
    "What was the baby’s white blood cell count?",
    "What follow-up care was recommended for the baby?",
    "How many patients had an arterial blood gas analysis?",
    "How many patients had a history of gastritis?",
    "How many patients experienced respiratory issues?",
    "How many patients were admitted to the ICU?",
    "What was the baby girl’s initial arterial blood gas pH?",
    "What was the result of the baby girl’s blood culture?",
    "Was the elderly female patient eventually transferred back to the ICU?",
    "Did the baby girl pass the hearing screening?",
    "What was the baby's initial treatment for suspected infection?",
    "What was the baby girl’s gestational age at birth?",
    "Did the baby girl have any cardiovascular issues?"
]


ground_truths = [
    ["Patient recorded as having No Known Allergies to Drugs."],
    ["Yes, the elderly female patient had a prior 70 pack year smoking history."],
    ["Glucose-159."],
    ["2615 grams."],
    ["Yes."],
    ["One dose of surfactant."],
    ["Yes."],
    ["8"],
    ["Follow-up appointment with pediatrician within two days of discharge from the hospital."],
    ["2/Two patients"],
    ["1/One patient"],
    ["2/Two patients"],
    ["1/One patient"],
    ["7.37."],
    ["Negative."],
    ["Yes, she did."],
    ["Yes, she passed the hearing screening."],
    ["Ampicillin and gentamicin."],
    ["28 weeks and 2 days."],
    ["No."]
]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('826txt')#19m44s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#629
from datasets import Dataset
questions= [
    "Does the patient have any known drug allergies?",
    "How often does the patient consume alcohol?",
    "What symptoms should prompt the patient to call the doctor's office or seek immediate medical attention?",
    "What are the current findings regarding the chest tubes?",
    "Are there any new lung findings on the recent imaging?",
    "What should the patient do regarding tube feeds?",
    "Did the patient take imaging studies?",
    "What is the patient's heart rate on admission?",
    "What are the patient's vital signs on admission?",
    "Was the patient in distress on admission?",
    "How long should the patient remain NPO with tube feeds?",
    "How long was the patient on antibiotics?",
    "How many patients took Dilantin?",
    "How many patients took Keppra?",
    "What was observed in the patient's head CT scan?",
    "What procedure did the patient have for the CBD stone?",
    "How many patients were admitted to ICU?",
    "What were the patient's symptoms that led to the CT scan?",
    "What medication should the patient continue to take as an outpatient for heart health?",
    "What was the patient’s LDL level?"
]


ground_truths = [
    ["No,patient recorded as having no known allergies to drugs."],
    ["Five times per week(+EtOH 5x/wk)."],
    ["Fever, chest pain, shortness of breath, nausea, vomiting, constipation, abdominal pain, diarrhea."],
    ["Two left-sided chest tubes are in unchanged position. A tiny left apical pneumothorax is stable in appearance. A small left-sided pleural effusion and atelectasis within the left lower lobe are unchanged."],
    ["No new opacities are seen within the lungs."],
    ["The patient should remain NPO (nothing by mouth) and continue with tube feeds for the next 3 weeks, as per VNA instructions."],
    ["Yes."],
    ["121 bpm."],
    ["Temperature 97°F, heart rate 121 bpm, blood pressure 190/90 mmHg, respiratory rate 18, oxygen saturation 96% on 2L oxygen."],
    ["Yes"],
    ["3 weeks."],
    ["1"],
    ["1"],
    ["Atrophy, no acute changes."],
    ["Cholangioscopy"],
    ["2"],
    ["Left hemiparesis involving face, arm, and leg; seizure involving left eye deviation and movement of the left arm."],
    ["Aspirin and beta blocker."],
    ["2"],
    ["LDL was 104."]
]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('629txt')#13m

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
#615
from datasets import Dataset
questions= [
    "What was the patient's initial blood pressure?",
    "What caused the patient's motor vehicle collision?",
    "What was the patient's GCS score upon arrival?",
    "What were the injuries sustained by the patient?",
    "Which ribs were fractured?",
    "What were the results of the CT head scan for patient 2?",
    "How many liters of crystalloid fluid were given?",
    "Was the patient's splenic hilum intact?",
    "What was the dosage of Zoloft prescribed?",
    "Was the patient initially intubated?",
    "Was the patient advised to follow up with a sleep clinic?",
    "Was the patient advised to have a repeat video swallow study?",
    "How many patients had a diagnosis of sleep apnea?",
    "How many patients had a history of diabetes?",
    "What were the findings of the patient’s sleep study?",
    "What injuries did the patient sustain in the motor vehicle collision?",
    "Was the patient wearing a seatbelt at the time of the collision?",
    "What were the patient’s vital signs upon arrival at the hospital?",
    "What did the CT scan show?",
    "How often does the patient drink alcohol?"
]


ground_truths = [
    ["82/palpable."],
    ["The patient fell asleep at the wheel."],
    ["The patient arrived with a GCS of 5."],
    ["The patient sustained a grade III/IV splenic laceration, small left pneumothorax, pulmonary contusions, left rib fractures (ribs eight, nine, and ten), small pelvic rami superior and inferior fracture with intramuscular hematoma."],
    ["Ribs eight, nine, and ten."],
    ["The patient's official CT read showed a splattered spleen with hemoperitoneum, grade III/IV splenic laceration, the hilum appeared to be intact."],
    ["6"],
    ["Yes"],
    ["Zoloft 50 mg p.o. once daily."],
    ["No"],
    ["Yes"],
    ["Yes, she was advised to have a repeat video swallow study."],
    ["1"],
    ["1"],
    ["The sleep study indicated significant nocturnal hypoventilation and hypoxemia, with evidence of severe chemoreflex-mediated sleep-disordered breathing."],
    ["The patient sustained a grade III/IV splenic laceration, small left pneumothorax, pulmonary contusions, left rib fractures of ribs eight, nine, and ten, and small pelvic rami superior and inferior fracture with intramuscular hematoma."],
    ["No, she was not wearing a seatbelt."],
    ["She was afebrile. The vital signs were stable, although the blood pressure was elevated at 200/90."],
    ["The patient's official CT read showed a splattered spleen with hemoperitoneum, grade III/IV splenic laceration, the hilum appeared to be intact."],
    ["The patient drinks three to four drinks two to three times per week."]]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('615txt')#19m42s

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#601
from datasets import Dataset
questions= [
    "What is the patient's age?",
    "What are the patient's past medical histories?",
    "What did CXR show?",
    "Why was she transferred to the CCU?",
    "What did the patient transfer to the CCU for?",
    "What was discovered during the patient's recent admission for jaw and chest pain?",
    "What medications was the patient discharged with after the recent admission for jaw and chest pain?",
    "What was the patient's condition upon her visit to the cardiology clinic?",
    "What was the patient's heart rhythm and rate in the medicine floor telemetry?",
    "What intervention was attempted in the CCU and what was the result?",
    "What were the patient's discharge medications?",
    "How many times a day was Lactulose prescribed?",
    "How many tests indicated signs of myocardial injury?",
    "Does the patient have a history of heart conditions?",
    "Does the patient have a history of chronic diarrhea?",
    "What was the patient’s WBC count?",
    "How many patients have a history of MRSA?",
    "Does the patient have a history of chronic diarrhea?",
    "What was the estimated right atrial pressure?",
    "How many patients took EEG?"
]


ground_truths = [
    ["79."],
    ["A history of rheumatic heart disease, rapid atrial fibrillation (AFib), chronic obstructive pulmonary disease (COPD), and a significant smoking history."],
    ["CXR showed pulmonary edema."],
    ["The patient sustained a grade III/IV splenic laceration, small left pneumothorax, pulmonary contusions, left rib fractures (ribs eight, nine, and ten), small pelvic rami superior and inferior fracture with intramuscular hematoma."],
    ["The patient was transferred to the CCU for further monitoring and management."],
    ["A 40% proximal LAD lesion."],
    ["Diltiazem."],
    ["She was in rapid a-fib, with a rate near 150; admission for cardioversion was discussed at length, but the patient refused, so she was started on metoprolol 50mg, aspirin, and warfarin."],
    ["The telemetry showed AFib with rates to 50's, with periods of asystole lasting for a couple of seconds, during which the pt reported dizziness and nausea."],
    ["The plan at the CCU was to place a temporary pacer wire so that rate control therapy could be safely started once a back-up rhythm could be guaranteed by the temporary pacer. Venous cannulation was unable to be achieved at the bedside. However, during the attempts to place the venous cannula, the pt converted to sinus rhythm at 70-80 bpm."],
    ["Metoprolol, aspirin, and warfarin."],
    ["3"],
    ["2"],
    ["Yes"],
    ["Yes"],
    ["WBC-11.3"],
    ["1"],
    ["Yes"],
    ["The estimated right atrial pressure is 11-15mmHg."],
    ["1"]
                 ]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('601txt')#10m47s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#563
from datasets import Dataset
questions= [
    "What is the patient's age?",
    "What were her vital signs in the emergency room?",
    "What was the patient’s systolic blood pressure on arrival?",
    "Is the patient scheduled to follow up with a pulmonologist?",
    "Did the patient receive medications in the emergency room?",
    "Did the patient take imaging studies?",
    "Was imaging done to assess the patient's lung condition?",
    "Did the patient report any symptoms of fever or chills?",
    "What was the patient’s oxygen saturation upon discharge?",
    "Was there any mention of smoking in the patient’s history?",
    "Did the patient have a history of smoking?",
    "How many patients had a history of diabetes mellitus?",
    "How many patients had a history of smoking?",
    "How many patients had a history of heart disease or related surgeries?",
    "How many patients were treated for pneumonia?",
    "How many patients had a history of hypertension?",
    "Was the patient’s ECG monitored?",
    "What were the findings on the MRI?",
    "How many episodes of hypotension did the patient experience?",
    "What is the patient's platelet count?"
]

ground_truths = [
    ["55"],
    ["Temperature 97.3°F, heart rate 109, blood pressure 101/69, respiratory rate 20, oxygen level 98% on 6 liters of oxygen, 94% on room air."],
    ["101 mmHg"],
    ["Yes, she is scheduled to follow up with a pulmonologist."],
    ["Yes, she was treated with Combivent nebulizations three times, received 5 liters of normal saline intravenously, was given Levofloxacin 500 mg intravenously for suspected pneumonia, and received Solumedrol 125 mg intravenously to reduce airway inflammation."],
    ["Yes"],
    ["Yes, chest X-ray was performed."],
    ["No, the patient denied having fever or chills."],
    ["100% 2L NC."],
    ["Yes"],
    ["Yes"],
    ["2"],
    ["1"],
    ["2"],
    ["2"],
    ["2"],
    ["Yes"],
    ["MRI showed a systolic artery calcified with a multilobar aspiration pneumonia."],
    ["2"],
    ["62"]
                 ]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('563txt')#8m51s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#417
from datasets import Dataset
questions= [
    "What is the age and gender of the patient with COPD?",
    "What are the patient’s known medical histories?",
    "What is the patient’s vital signs on presentation?",
    "What were the patient's main symptoms upon presenting to the Emergency Department?",
    "What is the diagnosis for this patient?",
    "What did the patient's chest X-ray show?",
    "Does the patient have a history of smoking?",
    "What is the patient’s platelet count?",
    "What medications were prescribed upon discharge?",
    "What is the age and gender of the patient with TTP?",
    "What was the patient's temperature in the Emergency Department?",
    "How many days was the patient initially started on Levaquin for community-acquired pneumonia?",
    "Did the patient have a fever upon admission?",
    "What was the patient’s respiratory rate upon admission?",
    "How many patients had a history of anemia?",
    "How many patients were treated with medications for nausea?",
    "How many patients had a history of chronic illness (e.g., COPD, diabetes)?",
    "How many patients took a multivitamin?",
    "How many patients had a smoking history?",
    "How many patients had a fever?"
]

ground_truths = [
    ["The patient is an 89-year-old male."],
    ["89-year-old male with a history of chronic obstructive pulmonary disease and ITP."],
    ["Temperature 100.5°F, blood pressure 99/50, heart rate 126, respiratory rate 34 (decreasing to 22 with nebulizer treatment), O2 saturation 94% on 2 liters."],
    ["He presented to the Emergency Department after a few hour history of chest and abdominal discomfort, increasing shortness of breath, and nausea with an episode of vomiting x1. He notes chest pressure with radiation to the back into the left arm, severity and associated epigastric discomfort with nausea and vomiting x1 in the Emergency Department."],
    ["Pneumonia, COPD exacerbation, acute myocardial infarction, hypotension.Famotidine, Fosamax, calcium, vitamin D, and prednisone."],
    ["Left lower lung zone opacity, mild congestive heart failure."],
    ["Yes, approximately 70 pack-year history of smoking."],
    ["116."],
    ["Acetaminophen, Docusate sodium, Fluoxetine, Prilosec, Multivitamin, Olanzapine, Atorvastatin, Salmeterol, Metoprolol, Nitroglycerin, Combivent inhaler, Prednisone taper, Albuterol nebulizer treatments, Ipratropium nebulizer, Haldol, Enteric coated aspirin."],
    ["24-year-old female."],
    ["101.7"],
    ["14"],
    ["Yes, she had a fever with a temperature of 101.7°F"],
    ["42"],
    ["2"],
    ["2"],
    ["2"],
    ["1"],
    ["2"],
    ["2"]
                 ]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('417txt')#12m2s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [12]:
#403
from datasets import Dataset
questions= [
    "What is the patient's age?",
    "Does the patient have any known drug allergies?",
    "What was the temperature of the patient at nursing home?",
    "What findings were noted on the CXR?",
    "Who was scheduled for the patient's follow-up care?",
    "What measures were taken for the patient during his hospital stay?",
    "What was the patient's temperature in the ER?",
    "What was the patient's tobacco, alcohol, and illicit drug use history?",
    "Did the patient have a fever?",
    "What should the patient do if they have worsening seizures after discharge?",
    "Does the patient have a history of tobacco use?",
    "Did the patient have a negative head CT?",
    "What type of rash does the patient have?",
    "How did the patient's mental status change after seizure treatment?",
    "Was there evidence of osteomyelitis in the patient's stage IV sacral decubitus ulcer?",
    "What was the plan for the patient's insulin therapy after hypoglycemia episodes?",
    "What did the patient's Doppler exam show about the IVC?",
    "Why was the patient intubated?",
    "What were the results of the patient’s sputum cultures?",
    "Were the patient’s seizures controlled?"
]

ground_truths = [
    ["64 years old"],
    ["No known drug allergies"],
    ["99.6°F"],
    ["CXR showed new LLL opacity."],
    ["neurologist Dr."],
    ["During his hospital stay his LFTs/bili and coags remained stable.  He underwent an abdominal U/S of liver w/ normal TIPS evaluation with wall-to-wall flow. No ascites identified.  He was continued on lactulose and rifaxamin."],
    ["98.9°F"],
    ["Remote tobacco history, no alcohol or illicit drug use"],
    ["Denies fever"],
    ["If they experiece any worsening of their seizure frequency, develop new types of seizures, develop changes in mental status, weakness, changes in sensation, vision, or language, and severe headaches, vertigo, or anything else that concerns you seriously."],
    ["Yes, remote tobacco history."],
    ["Yes, he had a neg head CT."],
    ["Maculopapular rash on back"],
    ["His mental status started to improve significantly."],
    ["No evidence of osteomyelitis per X-ray."],
    ["He was temporarily taken off home lantus as had episodes of hypoglycemia. He was restarted on his home dose of lantus without problem."],
    ["IVC demonstrates patency with triphasic waveforms."],
    ["He was intubated for airway protection"],
    ["Sputum cultures were positive for klebsiella, proteus, sensitive to meropenem, zosyn and tobra"],
    ["Yes."]
                 ]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('403txt')#13m40s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#371
from datasets import Dataset
questions= [
    "What are the discharge instructions for the patient?",
    "What surgery did the patient undergo?",
    "What medications is the patient prescribed?",
    "How was the patient's lung and heart status postoperatively?",
    "What conditions did patient get a history of?",
    "What type of aneurysm is described?",
    "What is the prescribed dose of Percocet?",
    "What is the diameter of the abdominal aortic aneurysm mentioned?",
    "What imaging technique showed no signs of CHF postoperatively?",
    "Where was the patient intubated postoperatively?",
    "What medication is given to prevent blood clots?",
    "How much Lasix is the patient taking daily?",
    "What is the patient's history of cardiovascular disease?",
    "What medication is prescribed for managing high blood pressure?",
    "What was noted about the patient’s skin during the physical examination?",
    "What is the purpose of the PICC line mentioned in the discharge instructions?",
    "What is the patient's HbA1c level?",
    "What did the bronchoscopy show?",
    "What did the right calf show?",
    "What is the dosage of Vancomycin administered?"
]

ground_truths = [["Patient will be discharged to home with instructions to followup with Dr.  in  weeks, and he will have a follow-up CT angiogram here in about one month."],
                 ["An endovascular repair for an abdominal aortic aneurysm."],
                 ["Atorvastatin 20 mg daily, Colchicine 0.6 mg daily, Digoxin 0.25 mg daily, Lasix 120 mg daily, Lopressor XL 150 mg daily, Moexipril 7.5 mg daily, Percocet 1-2 tablets every 4-6 hours as needed, Coumadin 5 mg daily at bedtime, Levaquin 500 mg daily for 10 days"],
                 ["The patient's pulmonary status improved after the diuresis, and patient subsequently underwent a bronch, which showed no plugging, no secretions, and no signs of CHF."],
                 ["Coronary artery disease, CHF,Abdominal aortic aneurysm"],
                 ["Abdominal aortic aneurysm."],
                 ["1-2 tablets every 4-6 hours as needed."],
                 ["Approximately 5.6 cm infrarenal in diameter."],
                 ["Bronchoscopy (bronch) showed no plugging, no secretions, and no signs of CHF."],
                 ["In the PACU"],
                 ["Coumadin."],
                 ["Lasix 120 mg p.o. q.d. daily"],
                 ["Coronary artery disease and CHF."],
                 ["Medications for managing high blood pressure: Vancomycin, Zosyn, and a dose of a drug called a clot."],
                 ["Dry, flaking feet."],
                 ["To administer medications and fluids."],
                 ["5.3%."],
                 ["No plugging, no secretions."],
                 ["MRI: 1.4 x 1.2 x 1.0 cm complex cystic lesion without definite flow within or around it. The lesion contains floating echogenic debris but no solid nodular component. The ankle mortise is preserved. There is no abnormal soft tissue gas."],
                 ["Vancomycin 1000 mg IV HD PROTOCOL"]]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('371txt')#13m

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#359
from datasets import Dataset
questions= [
    "How old is the patient?",
    "What surgery did the patient undergo?",
    "What is the patient's blood pressure?",
    "What allergies did the patient have?",
    "What imaging studies were performed on the patient?",
    "Has the patient got a history of diabetes",
    "Were there any problems after the operation?",
    "Why was the patient admitted for elective surgery again?",
    "What was the patient’s intraoperative blood loss?",
    "What medications was the patient taking before the recent admission?",
    "What was the patient’s INR level?",
    "What was the patient's respiratory rate?",
    "What was the patient's oxygen saturation on room air?",
    "What were the results of the Chem-7 panel?",
    "Was the patient febrile?",
    "What did the oropharynx examination reveal?",
    "What did the cardiac examination reveal?",
    "What did the chest X-ray show?",
    "Was the head CT scan result normal?",
    "What is the patient’s code status in the second case?"
]

ground_truths = [
    ['76'],
    ['Elective hip surgery, specifically an open reduction and internal fixation of the left hip, for revision due to nonunion of a previous hip fracture'],
    ['BP 156/70'],
    ['The patient is allergic to sulfa drugs, which cause a rash.'],
    ['Chest x-ray,CT scan of the head'],
    ['Yes'],
    ['No, there were no complications.'],
    ["Due to nonunion of the previous hip fracture."],
    ["900mL"],
    ["Atenolol 50 mg daily, perphenazine 7.5 mg twice daily, Tagamet, and aspirin."],
    ["28.4%."],
    ["1.3."],
    ["16"],
    ["96%."],
    ["Sodium: 141, Potassium: 4.7, Chloride: 110, Bicarbonate: 22, Creatinine: 0.4."],
    ["No, the patient remained afebrile."],
    ["The oropharynx was clear."],
    ["The cardiac exam showed an irregularly irregular rhythm and borderline tachycardia."],
    ["The chest X-ray showed ventricular enlargement and possible early CHF."],
    ["DNR/DNO"]]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('359txt')#18m

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [13]:
#239
from datasets import Dataset
questions= [
    "What are the patient's vital signs upon admission?",
    "What are the main medical conditions of the adult patient?",
    "What is the patient's blood pressure upon admission?",
    "Does the patient have any allergies?",
    "What are the patient's conditions?",
    "What was the infant's birth weight?",
    "What is the patient's current heart rate?",
    "What ventilation settings are used for the preterm infant on day five of life?",
    "How was the infant’s surfactant therapy administered?",
    "What is the management for severe mitral regurgitation?",
    "What is the recommended action for the patient with low blood sugar and heart failure?",
    "When did the infant switch back to conventional ventilation?",
    "What were the measurements recorded for the infant's head circumference at birth?",
    "What does the presence of harsh heart sounds in the adult patient suggest?",
    "What medication is the patient taking for anticoagulation?",
    "What is the oxygen saturation level of the patient?",
    "How many doses of surfactant did the infant receive?",
    "What are the patient’s bowel sounds described as?",
    "What medication should be stopped for the patient with low blood sugar?",
    "What was the patient’s temperature at right heart catheterization?"
]

ground_truths = [["BP 149/86, heart rate 72, respiratory rate 16, and saturations 100%, intubated on ventilator."],
                 ["The adult patient has a history of rheumatic heart disease, status post aortic valve replacement (AVR), mitral valve replacement (MVR), and coronary artery bypass grafting (CABG) of one vessel. The patient is also on Coumadin (warfarin)."],
                 ["129/83 mmHg"],
                 ["No known allergies."],
                 ["Intubated, young-appearing man attempting to pull at the ET tube with his left hand. HEENT: Nonicteric. Neck: Supple, no carotid bruits. Chest was clear to auscultation. Cardiovascular: Regular rate and rhythm, harsh S1 and S2 sounds, no murmur. Abdomen: Soft, nontender, positive bowel sounds. Extremities: No edema. Neurologically: Does not open eyes to voice or painful stimulation. Cranial nerves: Pupils| mm down to 1 bilaterally. EOMs full. Positive doll's eyes. Corneal reflexes: Absent bilaterally. Facial symmetry: ET restricts the lower face, but upper face appears wrinkling, symmetrically. Gag reflex: Gagging on the ET. Motor: Increased tone in all four extremities. Moves left side spontaneously, reaching and grabbing for the ET tube with the left hand. No spontaneous movement of the right hemibody. Decerebrate posturing of the right arm with pain and flexes knees and ankle with pain applied to both legs. Purposely withdraws, localizes with the left arm."],
                 ["1005 gm"],
                 ["92 beats per minute"],
                 ["Low ventilatory settings with pressures of 17/5, rate 22, FIO2 .3 to 0.4"],
                 ["He received three doses of surfactant."],
                 ["Severe MR"],
                 ["Stop taking your avandia as your blood sugar has been low and it is not the best medication for you considering your heart failure."],
                 ["On day five of life."],
                 ["27 cm, greater than 50th percentile."],
                 ["Harsh S1 and S2 sounds."],
                 ["Coumadin."],
                 ["100%"],
                 ["Three"],
                 ["Positive bowel sounds."],
                 ["avandia"],
                 ["97.5°F."]
]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('239txt')#10m47

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
#211
from datasets import Dataset
questions= [
    "What are the patient's vital signs upon admission?",
    "What medication was given for pain management in the elderly patient?",
    "What is the patient's blood pressure reading?",
    "Does the patient have any allergies?",
    "What follow-up was recommended for the patient after discharge?",
    "What imaging studies were performed on the patient?",
    "What was the patient's condition upon arrival to the trauma bay?",
    "How was the patient injured?",
    "Does the patient use tobacco or alcohol?",
    "What is the medical history of the patient's mother?",
    "What were the findings of the neurological exam?",
    "What were the lung exam findings?",
    "What were the results of the patient's C-spine, chest, abdomen, and pelvis CT?",
    "What was the neurosurgery consultation's recommendation?",
    "What were the results of the patient's cervical spine MRI?",
    "What was the recommendation for the patient's cervical collar?",
    "What were the discharge instructions for the patient?",
    "What was the status of the patient's mucous membranes (MM)?",
    "What was the treatment plan for the patient's cellulitis?",
    "What was the treatment plan for the patient’s urinary tract infection (UTI)?"
]

ground_truths = [["The patient was hypotensive with sbp in 80's, given 2 units of packed cells and 1 liter of fluids and pressure up to low 100's. Heart rate in the 90's. Afebrile, Lactate 1.8."],
                 ["Acetaminophen 325 mg Tablet, 1-2 Tablets PO Q4-6H (every 4 to 6 hours) as needed for pain."],
                 ["The patient was hypotensive with SBP in 80's, given 2 units of packed cells and 1 liter of fluids and pressure up to low 100's."],
                 ["No known allergies."],
                 ["Outpatient CT scan to be done in 2 weeks. Follow up in clinic in 2 weeks after CT scan is done. No need to follow up in trauma clinic but you may call the clinic at  with any questions."],
                 ["Pt taken for emergent head, C-spine, chest/abd/pelvis CT."],
                 ["On arrival to trauma bay pt with GCS of with waxing and waning of mental status."],
                 ["Struck by car"],
                 ["Occasional tobacco use (cigarettes) Denies EtOH use"],
                 ["Mother had gastric cancer."],
                 ["Neuro: alert, mild distress, following commands, GCS"],
                 ["Lungs: CTA bilat, equal BS"],
                 ["CT of C-spine, chest/abd/pelvis all negative."],
                 ["Neurosurgery consulted and pt loaded on Dilantin and started on Dilantin 100 IV TID."],
                 ["MRI of C-spine showed no fractures or ligamentous injuries."],
                 ["C-collar d/c'ed."],
                 ["Return to emergency room if having severe headaches or headaches not controlled with pain meds, if having excessive sleepiness or changes in mental status, avoid any physical activity that may put you at risk for repeat head injury, no driving while taking Percocet"],
                 ["MM dry."],
                 ["Keflex for 14 days total, elevation of the RLE, and dry dressing changes."],
                 ["Cultures grew out pan-sensitive enterococcus. The Keflex being given for her cellulitis should prove able to clear this UTI as well."]

]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('211txt')#10m16s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#205
from datasets import Dataset
questions= [
    "What is the patient's history of coronary artery disease?",
    "What allergie did the patient have?",
    "What imaging studies were performed on the patient?",
    "What medication is the patient taking?",
    "What were the results of chest CT?",
    "What follow-up instructions were given to the patient?",
    "How much coffee ground material was found during the nasogastric lavage?",
    "What change in blood pressure was noted in the patient?",
    "How was the patient's nausea relieved before hospitalization?",
    "Did the patient have a history of peptic ulcer disease or gastroesophageal reflux disease?",
    "What were the patient’s vital signs at the time of examination?",
    "What did the neck examination reveal?",
    "What were the findings of the lung examination?",
    "What did the H. pylori test result indicate?",
    "What were the findings from the chest x-ray?",
    "What treatment was provided for the urinary tract infection?",
    "What medications were included in the patient’s discharge plan?",
    "What medications did the patient receive for his urinary tract infection?",
    "What was the patient's PT and PTT levels?",
    "What was noted about the patient's neck during the examination?"


]

ground_truths = [["He has a history of coronary artery disease status post coronary artery bypass graft and multiple stents."],
                 ["No known drug allergies."],
                 ["Chest and pelvic x-rays were normal. Head CT was negative. Abdominal CT was within normal limits. Chest CT showed a right pneumothorax."],
                 ["Methadone 5 mg po b.i.d. for five days only with Ibuprofen 400 mg t.i.d. Cefalexin 500 mg b.i.d. times four days. Pantoprazole 40 mg po q.d. Dilaudid 2 mg po q 6 hours prn for five days."],
                 ["Chest CT showed a right pneumothorax."],
                 ["The patient can follow up at the Trauma Clinic if there are any new developments with the wounds or problems relating to this injury. Otherwise he is to obtain a primary care physician for long term healthcare.The patient was instructed to call Dr. to set up a follow-up appointment in 4 weeks. The results of the blood test should be faxed to Dr. at .The patient needs to have a blood test done in about one week, and the results should be faxed to Dr. .The discharge plan includes moving to a Skilled Nursing Facility, which will be mentioned in the discharge addendum."],
                 ["One liter of coffee grounds which did not clear with lavage."],
                 ["His systolic blood pressures also dropped from 130 to 90"],
                 ["Zantac over-the-counter."],
                 ["He has no known history of peptic ulcer disease or gastroesophageal reflux disease."],
                 ["Temperature 98.1, blood pressure 118/62, heart rate 71, respirations 18, O2 saturation 99% on two liters."],
                 ["Right IJ in place, noted to be clean, dry and intact without erythema."],
                 ["Clear to auscultation bilaterally with the exception of decreased breath sounds at the basis."],
                 ["H. pylori negative."],
                 ["Small bilateral pleural effusions and atelectasis with compression/collapse of the basilar left lung associated with an elevated left hemi-diaphragm."],
                 ["The patient was started on Levofloxacin 500 mg q day for a urinary tract infection secondary to coag negative staphylococcus."],
                 ["The patient’s discharge medications will be found on his page one form as well as on the discharge addendum."],
                 ["A liter of coffee grounds that did not clear with lavage."],
                 ["PT-11.6, PTT-23.5."],
                 ["Right IJ in place, noted to be clean, dry and intact without erythema."]]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('t5-205txt')#28m30.4

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
#198
from datasets import Dataset
questions= [
    "Why was the patient admitted to the hospital?",
    "What allergie did the patient have?",
    "What imaging studies were performed on the patient?",
    "What medication is the patient taking?",
    "What were the results of chest CT?",
    "What follow-up instructions were given to the patient?",
    "How did the patient get to the hospital?",
    "What treatment was given for the right pneumothorax?",
    "What is the patient's orientation status upon arrival?",
    "What was the patient's lactate level?",
    "What is the follow-up plan for the female patient?",
    "What antibiotic was prescribed at discharge?",
    "How many stab wounds did the male patient have?",
    "What is the patient's O2 saturation on a non-rebreather?",
    "What were the results of the patient's chest and pelvic x-rays?",
    "What was the patient's creatinine level?",
    "What was the female patient's heart rate upon recheck?",
    "What medication was the female patient given for DVT prophylaxis?",
    "How did the female patient react to the amiodarone bolus?",
    "What were the female patient's vital signs upon initial examination?"
]

ground_truths = [["He was stabbed in the back three times."],
                 ["No known drug allergies4."],
                 ["Chest and pelvic x-rays were normal. Head CT was negative. Abdominal CT was within normal limits. Chest CT showed a right pneumothorax."],
                 ["Methadone 5 mg po b.i.d. for five days only with Ibuprofen 400 mg t.i.d. Cefalexin 500 mg b.i.d. times four days. Pantoprazole 40 mg po q.d. Dilaudid 2 mg po q 6 hours prn for five days."],
                 ["Chest CT showed a right pneumothorax."],
                 ["The patient can follow up at the Trauma Clinic if there are any new developments with the wounds or problems relating to this injury. Otherwise he is to obtain a primary care physician for long term healthcare."],
                 ["EMS"],
                 ["A right-sided chest tube was placed without complications."],
                 ["The patient was alert and oriented times three."],
                 ["Lactate level was 0.2."],
                 ["The follow-up plan for the female patient is to continue treatments as described and undergo rehabilitation for unit myopathy. She should follow up with Dr. within 6-8 weeks."],
                 ["Cefalexin 500 mg BID for four days."],
                 ["Three stab wounds."],
                 ["100%."],
                 ["Normal."],
                 ["0.9."],
                 ["101."],
                 ["Subcutaneous Heparin."],
                 ["The female patient returned to normal sinus rhythm (NSR) shortly thereafter."],
                 ["Temperature 97.4, pulse 101, blood pressure 205/76, respirations 18, O2 saturation 98% on room air. Rechecked blood pressure was 166/58."]



]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('t5-198txt')#17m

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:

#167
from datasets import Dataset
questions= [
    "What is the patient's primary diagnosis?",
    "What medications is the patient currently taking for maintenance?",
    "Has the patient got a history of drug?",
    "What allergies did the patient have?",
    "What were the results of the patient's chest X-ray on admission?",
    "How many loose stools does the patient report per day?",
    "What is the patient presenting for?",
    "Does the patient have a fever or chills?",
    "What medication is prescribed as needed for anxiety?",
    "What is the patient's temperature on admission?",
    "What is the patient's heart rate on admission?",
    "What was the patient's oxygen saturation on room air upon admission?",
    "What were the heart examination findings on admission?",
    "What were the EKG findings on admission?",
    "What were the patient's white blood cell count preoperatively?",
    "How many patients have records of taking paracetamol?",
    "What will the patient follow up with?",
    "What were the medications prescribed for the 31-year-old female patient postoperatively?",
    "What is the patient's primary reason for presenting preop for a liver transplant?",
    "What was the patient's respiratory rate on admission for the liver transplant?"
]

ground_truths = [["Hepatitis C virus cirrhosis and right lobe hepatocellular carcinoma."],
                 ["Methadone 145 mg q. d.|Lives in .|History of alcohol abuse.  Denies any cigarette smoking.|On admission, temperature 98.1, heart rate 65, blood pressure 122/84 with a respiratory rate of 18,|8% on room air.  He was alert and oriented, no acute distress.  Lungs clear to auscultation bilaterally.  Heart regular rate and rhythm.  Abdomen soft, obese, positive bowel sounds, nonfocal abdomen, tender, negative rebound, negative hemorrhoids, positive umbilical hernia.  Rectal, guaiac negative, no hemorrhoids.  Extremities, positive venous changes and no edema.|On admission, the patient had a chest x-ray that showed no acute process, no suspicious nodules.  EKG on , sinus bradycardia, rate 49, inferior lateral flat T-waves.  On , CT of the thorax was stable, no lesions.|The patient was admitted to the transplant service.  He was made NPO after midnight.  IV fluid was started.  Lab work was sent off to use preop for the OR. Labs preop:  White count 3.7, hematocrit"],
                 ["Yes, he had a history of alcohol and IV drug abuse."],
                 ["Penicillin"],
                 ["The chest x-ray showed no acute process, no suspicious nodules."],
                 ["x10 per 24 hours"],
                 ["The 51-year-old male is presenting preop for a liver transplant in the a.m. The 41-year-old male was admitted to an outside hospital for new onset of tonic-clonic seizures. The 31-year-old woman is presenting for gastric bypass surgery (Roux-en-Y gastrojejunostomy)."],
                 ["No, the patient does not have a fever or chills."],
                 ["Ativan 1 mg p.r.n."],
                 ["98.1°F."],
                 ["65 beats per minute."],
                 ["98%."],
                 ["Heart regular rate and rhythm."],
                 ["Sinus bradycardia, rate 49, inferior lateral flat T-waves."],
                 ["8.1"],
                 ["0"],
                 ["The patient will follow up with their primary care physician next week to check liver function tests. The patient is scheduled to follow up with Dr.  next week, who is their infectious disease specialist. Additionally, the patient is to follow up in the clinic on  with Dr. ."],
                 ["Postoperatively, the patient was prescribed Roxicet Elixir for pain management, Zantac Elixir, Actigall, and Vitamin B12."],
                 ["He reports loose stools x10 per 24 hours associated with chronic abdominal pain, no fever or chills, no nausea or vomiting, no cough, no urinary frequency, or symptoms of UTI. Does admit to feeling hungry and has baseline shortness of breath."],
                 ["18 breaths per minute."]
]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)
dataset.save_to_disk('167txt')#11m35.7s

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
from datasets import Dataset
questions= [
    "Has the patient got a history of diabetes?",
    "What allergies did the patient have?",
    "Does the patient use tobacco or alcohol?",
    "What medication is the patient taking?",
    "What measures were taken for the patient?",
    "What imaging studies were performed on the patient?",
    "What follow-up care was scheduled for the patient?",
    "What did C-spine show?",
    "Does the patient use alcohol?",
    "Does the patient use cigarettes or alcohol?",
    "Did patient use tobacco?"
]

ground_truths = [["No."],
                 ["The patient has no known allergies to drugs."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes) but denies alcohol use."],
                 ["Dilantin 100 IV TID."],
                 ["Intubation and placement of a left radial arterial line for close blood pressure monitoring with a goal of keeping systolic blood pressure under 140. head CT, C-spine, chest/abdomen/pelvis CT to assess injuries. The patient was started on Dilantin 100 IV TID. Monitoring in the Trauma Intensive Care Unit (TICU). Repeat head CT and gradual weaning. Discontinuation of the Foley catheter and intravenous lines.Transfer from the ICU to the general hospital floor.  MRI of the C-spine to evaluate for potential injuries, which showed no fractures or ligamentous injuries, leading to clearance of the C-spine and discontinuation of the C-collar.Evaluation by Neurosurgery and Physical Therapy, deeming the patient fit for discharge with instructions for home care and follow-up."],
                 ["The patient underwent head CT, C-spine, chest/abd/pelvis CT scans, and an MRI of the C-spine"],
                 ["An outpatient CT scan was scheduled to be done in 2 weeks, and a follow-up clinic appointment was scheduled in 2 weeks after the CT scan.No need to follow up in the trauma clinic but the patient may call the clinic with any questions."],
                 ["The MRI of the C-spine showed no fractures or ligamentous injuries."],
                 ["No, the patient denies alcohol use."],
                 ["The patient occasionally uses cigarettes and denies alcohol use."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes)."]

]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)


In [12]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 11
})

In [13]:
dataset.save_to_disk('JSL-MedLlama-3-8B-v2.0-dataset')

Saving the dataset (1/1 shards): 100%|██████████| 11/11 [00:00<00:00, 618.31 examples/s]


In [14]:
ragas_input_df = dataset.to_pandas()
display(ragas_input_df.head())

,question,answer,contexts,ground_truths
0,Has the patient got a history of diabetes?,"Yes, the patient has a history of diabetes.",[Patient recorded as having No Known Allergies...,[No.]
1,What allergies did the patient have?,Allergic to,[Patient recorded as having No Known Allergies...,[The patient has no known allergies to drugs.]
2,Does the patient use tobacco or alcohol?,Answer: The patient occasionally uses tobacco ...,[Patient recorded as having No Known Allergies...,"[Yes, the patient occasionally uses tobacco (c..."
3,What medication is the patient taking?,The answer is Dilantin.,[Patient recorded as having No Known Allergies...,[Dilantin 100 IV TID.]
4,What measures were taken for the patient?,The answer is: The patient was given Dilantin ...,[Patient recorded as having No Known Allergies...,[Intubation and placement of a left radial art...


In [15]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [18]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)
     

In [16]:


def evaluate_ragas_dataset(dataset):
  result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
    
  )
  return result

In [19]:
qa_result = evaluate_ragas_dataset(dataset)

evaluating with [context_precision]


100%|██████████| 1/1 [01:02<00:00, 62.34s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:04<00:00,  4.83s/it]


evaluating with [answer_relevancy]


...

100%|██████████| 1/1 [00:02<00:00,  2.83s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:05<00:00,  5.10s/it]


evaluating with [answer_similarity]


100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


In [20]:
print(qa_result)

{'context_precision': 0.2727, 'faithfulness': 0.7273, 'answer_relevancy': 0.8613, 'context_recall': 1.0000, 'context_relevancy': 0.1193, 'answer_correctness': 0.8750, 'answer_similarity': 1.0000}


In [21]:
df = qa_result.to_pandas()

In [22]:
df.to_csv("JSL-MedLlama-3-8B-v2.0-dataset_ragas_result.csv")